In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [6]:
# MNIST datasets (images and labels)
train_dataset = torchvision.datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='data', train=False, transform=transforms.ToTensor())

# Data loaders (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


In [12]:
# CNN Model Architecture
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [15]:
# Instantiate model
cnn = ConvNet(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

In [16]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1087
Epoch [1/5], Step [200/600], Loss: 0.1298
Epoch [1/5], Step [300/600], Loss: 0.0612
Epoch [1/5], Step [400/600], Loss: 0.0355
Epoch [1/5], Step [500/600], Loss: 0.0268
Epoch [1/5], Step [600/600], Loss: 0.0401
Epoch [2/5], Step [100/600], Loss: 0.0949
Epoch [2/5], Step [200/600], Loss: 0.0615
Epoch [2/5], Step [300/600], Loss: 0.0252
Epoch [2/5], Step [400/600], Loss: 0.0364
Epoch [2/5], Step [500/600], Loss: 0.0469
Epoch [2/5], Step [600/600], Loss: 0.0441
Epoch [3/5], Step [100/600], Loss: 0.0408
Epoch [3/5], Step [200/600], Loss: 0.0572
Epoch [3/5], Step [300/600], Loss: 0.0092
Epoch [3/5], Step [400/600], Loss: 0.0181
Epoch [3/5], Step [500/600], Loss: 0.1108
Epoch [3/5], Step [600/600], Loss: 0.0035
Epoch [4/5], Step [100/600], Loss: 0.0303
Epoch [4/5], Step [200/600], Loss: 0.0148
Epoch [4/5], Step [300/600], Loss: 0.0057
Epoch [4/5], Step [400/600], Loss: 0.0016
Epoch [4/5], Step [500/600], Loss: 0.0101
Epoch [4/5], Step [600/600], Loss:

In [18]:
# Test the model

# Eval puts module into testing mode
# BatchNorm and Dropout are switched into a continuous form as opposed to a batch form
cnn.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 99.01 %
